In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, confusion_matrix
data = pd.read_csv("learn.csv")

In [2]:
import numpy as np
from catboost import Pool

In [3]:
del data ["Unnamed: 0"]

In [11]:
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [6]:
data=data[['loan_limit','loan_rate','credit_score',
           'company_enter_month','desired_amount','income_type','insert_time',
           'user_id','application_id', 'is_applied']]

In [7]:
data['earnedincome']=(data['income_type']=='EARNEDINCOME')*1
del data ['income_type']

In [8]:
data.sort_values(by=['user_id','insert_time'])

,loan_limit,loan_rate,credit_score,company_enter_month,desired_amount,insert_time,user_id,application_id,is_applied,earnedincome
9184097,20000000.0,15.0,810.0,2022-01-01,15000000.0,2022-05-21 23:41:09,9,248576,0.0,1
9184098,16000000.0,7.5,810.0,2022-01-01,15000000.0,2022-05-21 23:41:09,9,248576,0.0,1
9184099,16000000.0,7.0,810.0,2022-01-01,15000000.0,2022-05-21 23:41:09,9,248576,0.0,1
9184100,16000000.0,6.5,810.0,2022-01-01,15000000.0,2022-05-21 23:41:09,9,248576,0.0,1
9184101,20000000.0,14.4,810.0,2022-01-01,15000000.0,2022-05-21 23:41:09,9,248576,0.0,1
...,...,...,...,...,...,...,...,...,...,...
7784593,28000000.0,6.9,900.0,2022-04-01,200.0,2022-05-10 20:40:52,879697,1543548,0.0,1
7784594,50000000.0,11.9,900.0,2022-04-01,200.0,2022-05-10 20:40:52,879697,1543548,0.0,1
7784627,3000000.0,5.4,900.0,2022-04-01,200.0,2022-05-10 20:40:52,879697,1543548,0.0,1
7784643,50000000.0,10.6,900.0,2022-04-01,200.0,2022-05-10 20:40:52,879697,1543548,0.0,1


In [12]:
user=data['user_id'].values
time=data['insert_time'].dt.date.values
application=data['application_id'].values
arr=np.zeros(len(data))
recent_user=0
recent_application=0
i = 0
store = float(0)

In [13]:

for u in user:
    if u==recent_user:
        if application[i]==recent_application:
            arr[i]=store
        else:
            store=0
            for l in time_list:
                if time[i]==l:
                    store=store+1
                else:
                    store=store+0.9**((time[i]-l).days)
                    
            arr[i]=store
            time_list.append(time[i])
            recent_application=application[i]
    else:
        time_list=[]
        store=0
        arr[i]=store
        time_list.append(time[i])
        recent_user=user[i]
        recent_application=application[i]
    i=i+1

In [14]:
data['recentappcount'] = arr

In [15]:
data=data[data['insert_time']<'2022-06-01']
data=data.sort_values(by='is_applied')
data=data.drop_duplicates(['application_id'],keep='last')

In [16]:
train_data=data[data['insert_time']<'2022-05-16']
train_data2=train_data.copy()
del train_data['is_applied']
test_data=data[data['insert_time']>='2022-05-16']
test_data=test_data[test_data['insert_time']<'2022-06-01']
test_data2=test_data.copy()
del test_data['is_applied']
train_label=train_data2['is_applied']
test_label=test_data2['is_applied']

In [17]:
model = CatBoostClassifier(iterations=4,
                           depth=15,
                           learning_rate=0.83,
                           loss_function='Logloss',
                           verbose=True)

In [18]:
model.fit(train_data,train_label)

0:	learn: 0.5935547	total: 454ms	remaining: 1.36s
1:	learn: 0.5742657	total: 918ms	remaining: 918ms
2:	learn: 0.5653454	total: 1.25s	remaining: 418ms
3:	learn: 0.5617062	total: 1.61s	remaining: 0us


In [25]:
preds_proba=model.predict_proba(test_data)
preds_proba=preds_proba[:,1]

In [26]:
f1_score(test_label,preds_proba> 1- 0.5179389239664024)

0.7411930065571477

In [24]:
preds_class=model.predict(test_data)
f1_score(test_label,preds_class)

0.7357939647414523

In [81]:
confusion_matrix(test_label, preds_proba >  1- 0.5179389239664024).ravel()

array([35816, 26429, 14264, 58270])

In [30]:
model.get_feature_importance()

array([24.32075591, 23.08832804, 19.09794364,  2.41671911, 22.7779675 ,
        1.73469843,  0.61209716,  0.64020002,  5.3112902 ,  0.        ])

In [32]:
test_data.loc[:, "predict"] = preds_proba > 1- 0.5179389239664024

In [36]:
appyes = test_data.loc[test_data.predict, "application_id"]

In [38]:
newdata = pd.read_csv("learn.csv")

In [39]:
newdata.loc[:, "predict"] = 0

In [42]:
del newdata["Unnamed: 0"]

In [43]:
newdata.insert_time = pd.to_datetime(newdata.insert_time)
newdata.company_enter_month = pd.to_datetime(newdata.company_enter_month)

In [44]:
newdata=newdata[['loan_limit','loan_rate','credit_score',
           'company_enter_month','desired_amount','income_type','insert_time',
           'user_id','application_id', 'is_applied']]

In [46]:
newdata['earnedincome']=(newdata['income_type']=='EARNEDINCOME')*1
del newdata ['income_type']

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_13375/2673208710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['earnedincome']=(newdata['income_type']=='EARNEDINCOME')*1


In [47]:
xx = newdata.loc[newdata.application_id.isin(appyes)]

In [95]:
xx.loc[:, ["sum", "is_enough", "pred"]] = 0

In [51]:
xx = xx[["application_id", "loan_limit", "loan_rate", "is_applied", "desired_amount", "sum", "is_enough", "pred"]]

In [94]:
xx = xx.sort_values(by = ["application_id", "loan_rate"])

In [96]:
xx.iloc[0, 5] = xx.iloc[0, 1]

In [97]:
for i in range (1, len(xx)):
    xx.iloc[i, 5] = xx.iloc[i, 1]
    if (xx.iloc[i, 0] == xx.iloc[i-1, 0]):
        xx.iloc[i, 5] += xx.iloc[i-1, 5]

In [98]:
xx["is_enough"] = xx["sum"] > xx.desired_amount

In [99]:
for i in range(2, len(xx)):
    if (xx.iloc[i, 0] == xx.iloc[i-1, 0]):
        if (not xx.iloc[i, 6]):
            xx.iloc[i, 7] = 1
        elif (not xx.iloc[i-1, 6]):
            xx.iloc[i, 7] = 1
        elif (not xx.iloc[i-2, 6]):
            xx.iloc[i, 7] = 1
    else:
        xx.iloc[i, 7] = 1

In [100]:
xx.iloc[0, 7] = 1
xx.iloc[1, 7] = 1

In [101]:
xx

,application_id,loan_limit,loan_rate,is_applied,desired_amount,sum,is_enough,pred
8839935,59,100000000.0,4.0,1.0,500000000.0,100000000,False,1
8839923,59,24000000.0,4.1,0.0,500000000.0,124000000,False,1
8839918,59,60000000.0,4.3,0.0,500000000.0,184000000,False,1
8839955,59,3000000.0,5.5,0.0,500000000.0,187000000,False,1
8839926,59,50000000.0,6.7,0.0,500000000.0,237000000,False,1
...,...,...,...,...,...,...,...,...
9879289,2167719,25000000.0,17.7,1.0,3000000.0,25000000,True,1
9879290,2167719,25000000.0,19.1,0.0,3000000.0,50000000,True,0
8720895,2167790,14000000.0,12.8,0.0,10000000.0,14000000,True,1
8720858,2167790,42000000.0,15.5,0.0,10000000.0,56000000,True,0


In [102]:
f1_score(xx.is_applied, xx.pred)

0.4193164202067218

In [103]:
yesindex = xx.loc[xx.pred == 1].index

In [104]:
test = newdata.loc[newdata.insert_time >= "2022-05-16"]

In [105]:
test.loc[:, "predict"] = 0

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_13375/4117222034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:, "predict"] = 0


In [106]:
test.loc[yesindex, "predict"] = 1

In [107]:
test

,loan_limit,loan_rate,credit_score,company_enter_month,desired_amount,insert_time,user_id,application_id,is_applied,earnedincome,predict
8395658,13000000.0,17.5,650.0,2022-06-01,2000000.0,2022-05-16 00:11:50,355550,2136969,0.0,0,0
8395659,5000000.0,14.5,650.0,2022-06-01,2000000.0,2022-05-16 00:11:50,355550,2136969,0.0,0,0
8395660,3000000.0,12.5,650.0,2022-06-01,2000000.0,2022-05-16 00:11:50,355550,2136969,1.0,0,1
8395661,10000000.0,17.5,630.0,2020-10-01,5000000.0,2022-05-16 00:11:51,102740,824356,1.0,1,1
8395662,5000000.0,16.3,700.0,2022-06-01,1000000.0,2022-05-16 00:12:14,707400,993637,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10264381,3000000.0,18.0,610.0,1997-04-01,50000000.0,2022-05-31 23:54:27,717233,455157,0.0,1,0
10264382,3000000.0,17.5,610.0,1997-04-01,50000000.0,2022-05-31 23:54:27,717233,455157,0.0,1,0
10264383,5000000.0,18.4,610.0,1997-04-01,50000000.0,2022-05-31 23:54:27,717233,455157,0.0,1,0
10264384,3000000.0,14.5,610.0,1997-04-01,50000000.0,2022-05-31 23:54:27,717233,455157,0.0,1,0


In [108]:
f1_score(test.is_applied, test.predict)

0.3871856352196723

In [110]:
confusion_matrix(test.is_applied, test.predict).ravel()

array([1645075,  116835,   53126,   53692])